In [ ]:
import json
import random
import re
from tqdm import tqdm
random.seed(42)

In [ ]:
test_data_indices = []
batch_size = 100
N = 100
train_data = []
test_data  = []

with open("./nucc_discord.txt", "r") as f:
    dialogs = f.read().split("\n")

with open("./discord-dialogs.json", "r") as f:
    dialogs += json.load(f)

In [ ]:
def batch_random():
    return random.choice(range(0, len(dialogs), 100))

def sentence_preprocessing(sentence):
    flag = True
    
    # assure its not emoji/ channnel/ mention
    if sentence.startswith("<"):
        flag = False
    
    # message should begins with ...
    if not re.match(r'[亜-熙ぁ-んァ-ヶa-zA-Z]', sentence):
        flag = False
    
    # assure its not a command prefix
    if re.match(r'[!|?|@|/]', sentence[0:3]):
        flag = False
    
    # no url
    if re.search("http", sentence):
        flag = False
        
    sentence = re.sub(r'<(!|&|@|#|:|[0-9a-zA-Z])+>', "", sentence)
    sentence = re.sub(r'[亜-熙ぁ-んァ-ヶa-zA-Z]+[さん|君|ちゃん]', "あなたは", sentence)
    if sentence.endswith(" "):
        sentence = sentence[:-1]
        
    if sentence == "":
        flag = False
        
    if len(sentence) > 60:
        flag = False
    
    if flag:
        return sentence
    else:
        return False

In [ ]:
dialogs = [sentence_preprocessing(x) for x in tqdm(dialogs) if sentence_preprocessing(x)]
len(dialogs)

In [ ]:
def push_test_data():
    global test_data
    i = batch_random()
    test_data_indices.append(i)
    test_data += dialogs[i:i+N]

def collect_train_data():
    global train_data
    for i in range(0, len(dialogs), 100):
        if not i in test_data_indices:
            train_data += dialogs[i:i+N]

In [ ]:
for _ in range(N):
    push_test_data()

collect_train_data()
len(train_data), len(test_data)

In [ ]:
with open("./input/train.txt", "w") as f:
    f.write("\n".join(train_data))
    
with open("./input/test.txt", "w") as f:
    f.write("\n".join(test_data))